# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Traffic Light Recognition Classifier Combined MobileNet with FabLeNet



---
## Step 0: Load The Data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

show_debug=False

import math
def dist(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def draw_box(img, l1, l2, color=[255, 0, 0], thickness=2):
    cv2.line(img, (l1[0],l1[1]), (l1[2],l1[3]), color, thickness)
    cv2.line(img, (l1[2],l1[3]), (l2[2],l2[3]), color, thickness)
    cv2.line(img, (l2[0],l2[1]), (l2[2],l2[3]), color, thickness)
    cv2.line(img, (l1[0],l1[1]), (l2[0],l2[1]), color, thickness)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def get_lines(img):
    #gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    lab = cv2.cvtColor(img,cv2.COLOR_RGB2LAB)
    gray = lab[:,:,2]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
    gray = clahe.apply(gray)
    
    if show_debug:
        plt.imshow(gray, cmap='gray')
        plt.show()
    #gray = cv2.GaussianBlur(gray,(5, 5),0)

    
    # Define a kernel size and apply Gaussian smoothing
    #kernel_size = 5
    #

    # Define our parameters for Canny and apply
    low_threshold = 30
    high_threshold = 120
    #edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= 80) & (scaled_sobel <= 255)] = 255
    gray=binary_output
    #gray = cv2.GaussianBlur(gray,(5, 5),0)
    #cv2.Canny(gray, low_threshold, high_threshold)
    edges=gray
    #cv2.imwrite('edges_input.jpg',np.dstack((edges, edges, edges)))
    
    if show_debug:
        plt.imshow(edges, cmap='gray')
        plt.show()
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 50     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 30 #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
        min_line_length, max_line_gap)
    return lines


def filter_lines(lines):
    filtered_lines=[]
    if lines is not None:
    
        for line in lines:
            if line[0,1]<line[0,3]:
                line[0,1],line[0,3]=line[0,3],line[0,1]
                line[0,0],line[0,2]=line[0,2],line[0,0]
        lines2=lines[:,0,:]
        for line in lines2:
            d=dist(line[0],line[1],line[2],line[3])
            #print(line, d)
            if (abs(line[0]-line[2])*4<abs(line[3]-line[1])):
                for l2 in lines2:
                    #print(l2, dist(line[0],line[1],l2[0],l2[1]))
                    if line[0]<l2[0] and (l2[0]-line[0])>abs(l2[1]-line[1])*2 and (l2[2]-line[2])>abs(l2[3]-line[3])*2:
                        d2=dist(line[0],line[1],l2[0],l2[1])
                        d3=dist(line[2],line[3],l2[2],l2[3])
                        if (d2>d*0.3 and d2<d*0.6 and d3>d*0.3 and d3<d*0.6):
                            if show_debug: print(line, l2, d, d2)
                            filtered_lines.append(np.array([line,l2]))
                            #draw_box(res1,line,l2)
    #draw_lines(res1, lines)
    return np.array(filtered_lines)

def annotate_image(test1):
    test1 = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)
    res1=test1.copy()

    #gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    lines = get_lines(test1)

    if lines is not None:
        if show_debug:
            draw_lines(res1, lines, (0,255,0),4)

    #example:
    #good:
    #[417 138 419  99] 39.05124837953327
    #[434 138 435 101] 37.013511046643494
    #~17px wide, 39px tall, 0.3..0.6
    #not good:
    #[423 123 424 149] 26.019223662515376
    #[388 165 389 143] 22.02271554554524
    #[385 166 386 140] 26.019223662515376
    filtered_lines=filter_lines(lines)
    #print(filtered_lines)
    for l in filtered_lines:
        draw_box(res1,l[0],l[1])
    return res1

def annotate_image_simplebox(test1, box):
    img = test1 #cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)
    color=[255, 0, 0]
    thickness=2
    cv2.line(img, (box[0],box[1]), (box[2],box[1]), color, thickness)
    cv2.line(img, (box[2],box[1]), (box[2],box[3]), color, thickness)
    cv2.line(img, (box[2],box[3]), (box[0],box[3]), color, thickness)
    cv2.line(img, (box[0],box[3]), (box[0],box[1]), color, thickness)
    return img

def add_to_box(box,x,y):
    if (x<box[0]): box[0]=x
    if (y<box[1]): box[1]=y
    if (x>box[2]): box[2]=x
    if (y>box[3]): box[3]=y

#test1 = cv2.imread('train_0274.png')
#res1=annotate_image(test1)
#plt.figure(figsize=(12, 8))
#plt.imshow(res1)
#plt.show()

import os
dirname='training_ros/'

write_box=False
annot_with_hint=True

if write_box:
    for i in os.listdir(dirname):
        print(i)
        test1 = cv2.imread(dirname+i)
        res1=annotate_image(test1)
        #print('o_'+dirname+i)
        cv2.imwrite('box_'+dirname+i, cv2.cvtColor(res1, cv2.COLOR_RGB2BGR))

if annot_with_hint:
    hintfile=open('annot_hint.txt', 'r')
    resultcsv=open('annot_'+dirname+'annot.csv', 'w')
    resultcsv.write("\"image\",\"xmin\",\"ymin\",\"xmax\",\"ymax\",\"label\"\n")
    lines=hintfile.readlines()
    last_color='n'
    for line in lines:
        line=line.strip()
        sp1=line.split(',')
        if (len(sp1)>1): last_color=sp1[1]
        sp2=sp1[0].split('-')
        min_idx=int(sp2[0])
        max_idx=min_idx
        if len(sp2)>1: max_idx=int(sp2[1])
        for idx in range(min_idx,max_idx+1):
            print(idx, last_color)
            test1 = cv2.imread(dirname+'train_%04d.png'%idx)
            test1rgb = cv2.cvtColor(test1, cv2.COLOR_BGR2RGB)
            lines = get_lines(test1rgb)
            filtered_lines=filter_lines(lines)
            #print(filtered_lines)
            if (filtered_lines is not None) and len(filtered_lines)>0:
                box=filtered_lines[0][0]
                for l2 in filtered_lines:
                    for l in l2:
                        add_to_box(box,l[0],l[1])
                        add_to_box(box,l[2],l[3])
                print(box)
                filename= last_color+'_'+"train_%04d.png"%idx
                color_code=-1
                if last_color=='y': color_code=1
                elif last_color=='r': color_code=2
                elif last_color=='g': color_code=3
                
                resultcsv.write("\"{}\",{},{},{},{},{}\n".format(filename, box[0], box[1], box[2], box[3], color_code))
                res1=annotate_image_simplebox(test1rgb,box)
                cv2.imwrite('annot_'+dirname+filename, cv2.cvtColor(res1, cv2.COLOR_RGB2BGR))
        
    resultcsv.close()
print("ready.")

    

0 r
[418 101 435 139]
4 r
[420 101 435 140]
6 r
[417  99 435 138]
7 r
[417  95 435 137]
8 r
[417  97 435 141]
16 r
[417 100 435 139]
17 r
[417  99 435 139]
18 r
[417 100 435 140]
24 r
[418 101 435 139]
25 r
[418 100 434 140]
28 r
[417  99 435 145]
30 r
[418 104 435 140]
31 r
[417  98 435 139]
32 r
[417 100 435 138]
34 r
[419 100 435 140]
37 g
[417  99 435 139]
39 g
[418 101 435 138]
43 g
[418 103 435 139]
47 g
[417 101 435 141]
49 g
[419 102 435 142]
51 g
[420 104 435 141]
52 g
[420 101 435 142]
53 g
[420 102 435 143]
55 g
[419 103 435 143]
57 g
[420 103 435 143]
58 g
[420 103 435 144]
59 g
[419 104 435 143]
60 g
[420 103 435 145]
61 g
[419 105 435 144]
63 g
[419 105 435 145]
64 g
[419 107 435 147]
65 g
[417 106 435 147]
66 g
[418 106 435 147]
68 g
[417 107 435 147]
69 g
[417 106 433 146]
70 g
[417 105 432 147]
71 g
[416 105 432 146]
72 g
[416 104 432 146]
73 g
[414 103 431 147]
74 g
[416 104 431 147]
75 g
[413 104 430 146]
76 g
[415 105 429 147]
77 g
[413 105 431 147]
78 g
[411 105 42

In [170]:
resultcsv.close()